In [1]:
# Importing Selenium and all related modules
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

#Importing the libraries to get the webdriver
import requests
import wget
import zipfile
import os

# Importing other required libraries
import time
import pandas as pd
from datetime import date, timedelta
import random
from datetime import date, timedelta


# Getting the webdriver
To have always the latest version of the Chrome webdriver, we can directly check and download the latets version available. 

In [ ]:
# # get the latest chrome driver version number
# url = 'https://chromedriver.storage.googleapis.com/LATEST_RELEASE'
# response = requests.get(url)
# version_number = response.text

# # build the donwload url
# download_url = "https://chromedriver.storage.googleapis.com/" + version_number +"/chromedriver_mac64.zip"

# # download the zip file using the url built above
# latest_driver_zip = wget.download(download_url,'chromedriver.zip')

# # extract the zip file
# with zipfile.ZipFile(latest_driver_zip, 'r') as zip_ref:
#     zip_ref.extractall("../ChromeDriver_Path") # you can specify the destination folder path here
# # delete the zip file downloaded above
# os.remove(latest_driver_zip)

# Creating the webdriver instance
The Chrome webdriver instance is generated.

In [2]:
# __________________Defining the Chrome Driver Instance
# Creating a webdriver instance
options = Options()

# options.add_argument('--headless') -> You can activate this option if you want to watch the scraping process.
options.add_argument('--disable-gpu')  # Last I checked this was necessary.
driver = webdriver.Chrome(r"C:\ikhsan\UNAIR\MBKM\Bangkit\1. Capstone\chromedriver.exe", chrome_options=options)

C:\Users\INES\AppData\Local\Temp\ipykernel_6524\1025969745.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\ikhsan\UNAIR\MBKM\Bangkit\1. Capstone\chromedriver.exe", chrome_options=options)
C:\Users\INES\AppData\Local\Temp\ipykernel_6524\1025969745.py:7: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(r"C:\ikhsan\UNAIR\MBKM\Bangkit\1. Capstone\chromedriver.exe", chrome_options=options)


# Defining job and country/location and loading the driver

In [3]:
job_name = ""
country_name = "Indonesia"

#We create the corresponding job name in URL
job_url ="";
for item in job_name.split(" "):
    if item != job_name.split(" ")[-1]:
        job_url = job_url + item + "%20"
    else:
        job_url = job_url + item
        
#We create the  corresponding country name in URL
country_url ="";
for item in country_name.split(" "):
    if item != country_name.split(" ")[-1]:
        country_url = country_url + item + "%20"
    else:
        country_url = country_url + item

url = "https://www.linkedin.com/jobs/search?keywords={0}&location={1}&geoId=103644278&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0".format(job_url,country_url)

# Opening the url we have just defined in our browser
driver.get(url)

# Detecting elements in our Browser

## Knowing how many jobs results are there

In [4]:
#We find how many jobs are offered.
jobs_num = driver.find_element(By.CSS_SELECTOR,"h1>span").get_attribute("innerText")
if ',' in jobs_num and (jobs_num := jobs_num.replace(',', '')).isdigit():
    jobs_num = int(jobs_num)

#Here I choose manually a number of jobs, so it wont take that long:
jobs_num = 100;

## The loop is initialized.

In [5]:
#We create a while loop to browse all jobs. 
i = 2
while i <= int(jobs_num/2)+1:
    #We keep scrollind down to the end of the view.
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    i = i + 1
    print("Current at: ", i, "Percentage at: ", ((i+1)/(int(jobs_num/2)+1))*100, "%",end="\r")
    try:
        #We try to click on the load more results buttons in case it is already displayed.
        infinite_scroller_button = driver.find_element(By.XPATH, '//*[@id="main-content"]/section[2]/button')
        infinite_scroller_button.click()
        time.sleep(1)
    except:
        #If there is no button, there will be an error, so we keep scrolling down.
        time.sleep(0.1)
        pass

# Loading Basic Data

In [6]:
#We get a list containing all jobs that we have found.
job_lists = driver.find_element(By.CLASS_NAME,"jobs-search__results-list")
jobs = job_lists.find_elements(By.TAG_NAME,"li") # return a list

#We declare void list to keep track of all obtaind data.
job_title_list = []
company_name_list = []
location_list = []
date_list = []
job_link_list = []

#We loof over every job and obtain all the wanted info.
for job in jobs:
    #job_title
    job_title = job.find_element(By.CSS_SELECTOR,"h3").get_attribute("innerText")
    job_title_list.append(job_title)
    
    #company_name
    company_name = job.find_element(By.CSS_SELECTOR,"h4").get_attribute("innerText")
    company_name_list.append(company_name)
    
    #location
    location = job.find_element(By.CSS_SELECTOR,"div>div>span").get_attribute("innerText")
    location_list.append(location)
    
    #date
    date = job.find_element(By.CSS_SELECTOR,"div>div>time").get_attribute("datetime")
    date_list.append(date)
    
    #job_link
    job_link = job.find_element(By.CSS_SELECTOR,"a").get_attribute("href")
    job_link_list.append(job_link)


# Loading More Data

In [7]:
jd = [] #job_description
seniority = []
emp_type = []
job_func = []
job_ind = []
for item in range(len(jobs)):
    job_func0=[]
    industries0=[]
    # clicking job to view job details
    
    #__________________________________________________________________________ JOB Link
    
    try: 
        job_click_path = f'/html/body/div/div/main/section/ul/li[{item+1}]'
        job_click = job.find_element(By.XPATH,job_click_path).click()
    except:
        pass
    
    #Wait for a bit for the website to charge (again with a random behavior)
    time.sleep(1+random.random())
    
    #__________________________________________________________________________ JOB Description
    jd_path = '/html/body/div/div/section/div/div/section/div/div/section/div'
    try:
        jd0 = job.find_element(By.XPATH,jd_path).get_attribute('innerText')
        jd.append(jd0)
    except:
        jd.append(None)
        pass
    
    #__________________________________________________________________________ JOB Seniority
    seniority_path='/html/body/div/div/section/div/div/section/div/ul/li[1]/span'
    
    try:
        seniority0 = job.find_element(By.XPATH,seniority_path).get_attribute('innerText')
        seniority.append(seniority0)
    except:
        seniority.append(None)
        pass

    #__________________________________________________________________________ JOB Time
    emp_type_path='/html/body/div/div/section/div/div/section/div/ul/li[2]/span'
    
    try:
        emp_type0 = job.find_element(By.XPATH,emp_type_path).get_attribute('innerText')
        emp_type.append(emp_type0)
    except:
        emp_type.append(None)
        pass
    
    #__________________________________________________________________________ JOB Function
    function_path='/html/body/div/div/section/div/div/section/div/ul/li[3]/span'
    
    try:
        func0 = job.find_element(By.XPATH,function_path).get_attribute('innerText')
        job_func.append(func0)
    except:
        job_func.append(None)
        pass

    #__________________________________________________________________________ JOB Industry
    industry_path='/html/body/div/div/section/div/div/section/div/ul/li[4]/span'
    
    try:
        ind0 = job.find_element(By.XPATH,industry_path).get_attribute('innerText')
        job_ind.append(ind0)
    except:
        job_ind.append(None)
        pass
    
    print("Current at: ", item, "Percentage at: ", (item+1)/len(jobs)*100, "%",end="\r")

In [8]:
job_data = pd.DataFrame({
    'Date': date_list,
    'Company': company_name_list,
    'Title': job_title_list,
    'Location': location_list,
    'Description': jd,
    'Level': seniority,
    'Type': emp_type,
    'Function': job_func,
    'Industry': job_ind,
    'Link': job_link
})

job_data.tail()

,Date,Company,Title,Location,Description,Level,Type,Function,Industry,Link
170,2023-04-14,PT GITS Indonesia,Sales Manager,"Jakarta, Jakarta, Indonesia",None,None,None,None,None,https://id.linkedin.com/jobs/view/risk-managem...
171,2023-04-23,Shopee,Category Admin - Business Development (Contrac...,"Jakarta, Indonesia",None,None,None,None,None,https://id.linkedin.com/jobs/view/risk-managem...
172,2023-05-13,PT. Aarti Jaya,Akunting Staff,"Cimahi, West Java, Indonesia",None,None,None,None,None,https://id.linkedin.com/jobs/view/risk-managem...
173,2023-05-09,PT. Padi Mulia Jaya,Warehouse Admin,"Tangerang Selatan, Banten, Indonesia",None,None,None,None,None,https://id.linkedin.com/jobs/view/risk-managem...
174,2023-04-29,Asuransi Astra,Risk Management Staff,"Jakarta, Jakarta, Indonesia",None,None,None,None,None,https://id.linkedin.com/jobs/view/risk-managem...


In [9]:
job_data.to_csv("Output/ddbb_{0}_{1}_.csv".format(job_name,country_name))